In [2]:
from time import time
import cv2
import cvzone
from cvzone.FaceDetectionModule import FaceDetector
import os
from tqdm import tqdm

####################################
classID = 1  # 0 is fake and 1 is real
outputFolderPath = 'DataCollect'
confidence = 0.8
save = True
blurThreshold = 35  # Larger is more focus

debug = False
offsetPercentageW = 10
offsetPercentageH = 20
camWidth, camHeight = 640, 480
floatingPoint = 6
####################################

# Directory setup (also adjust class label)
videoDirectory = 'Dataset_real'
videoFiles = [f for f in os.listdir(videoDirectory) if f.endswith('.mp4')]

detector = FaceDetector()

for videoFile in tqdm(videoFiles, desc="Processing Videos"):
    videoPath = os.path.join(videoDirectory, videoFile)
    cap = cv2.VideoCapture(videoPath)
    cap.set(3, camWidth)
    cap.set(4, camHeight)

    while True:
        success, img = cap.read()
        if not success:
            break  # Exit loop if video ends or error occurs

        imgOut = img.copy()
        img, bboxs = detector.findFaces(img, draw=False)
        listBlur = []  # True False values indicating if the faces are blur or not
        listInfo = []  # The normalized values and the class name for the label txt file

        if bboxs:
            for bbox in bboxs:
                x, y, w, h = bbox["bbox"]
                score = bbox["score"][0]

                if score > confidence:
                    offsetW = (offsetPercentageW / 100) * w
                    x = int(x - offsetW)
                    w = int(w + offsetW * 2)
                    offsetH = (offsetPercentageH / 100) * h
                    y = int(y - offsetH * 3)
                    h = int(h + offsetH * 3.5)

                    if x < 0: x = 0
                    if y < 0: y = 0
                    if w < 0: w = 0
                    if h < 0: h = 0

                    imgFace = img[y:y + h, x:x + w]
                    if debug:
                        cv2.imshow("Face", imgFace)
                    blurValue = int(cv2.Laplacian(imgFace, cv2.CV_64F).var())
                    listBlur.append(blurValue > blurThreshold)

                    ih, iw, _ = img.shape
                    xc, yc = x + w / 2, y + h / 2
                    xcn, ycn = round(xc / iw, floatingPoint), round(yc / ih, floatingPoint)
                    wn, hn = round(w / iw, floatingPoint), round(h / ih, floatingPoint)

                    if xcn > 1: xcn = 1
                    if ycn > 1: ycn = 1
                    if wn > 1: wn = 1
                    if hn > 1: hn = 1

                    listInfo.append(f"{classID} {xcn} {ycn} {wn} {hn}\n")

                    cv2.rectangle(imgOut, (x, y, w, h), (255, 0, 0), 3)
                    cvzone.putTextRect(imgOut, f'Score: {int(score * 100)}% Blur: {blurValue}', (x, y - 0), scale=2, thickness=3)
                    if debug:
                        cv2.rectangle(img, (x, y, w, h), (255, 0, 0), 3)
                        cvzone.putTextRect(img, f'Score: {int(score * 100)}% Blur: {blurValue}', (x, y - 0), scale=2, thickness=3)

            if save and all(listBlur) and listBlur:
                timeNow = time()
                timeNow = str(timeNow).split('.')
                timeNow = timeNow[0] + timeNow[1]
                cv2.imwrite(f"{outputFolderPath}/{timeNow}.jpg", img)
                for info in listInfo:
                    with open(f"{outputFolderPath}/{timeNow}.txt", 'a') as f:
                        f.write(info)

        if debug:
            cv2.imshow("Image", imgOut)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()

cv2.destroyAllWindows()


I0000 00:00:1711381334.929079  477423 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
Processing Videos: 100%|██████████| 50/50 [02:16<00:00,  2.74s/it]
